In [1]:
from unsloth import tokenizer_utils
def do_nothing(*args, **kwargs):
    pass
tokenizer_utils.fix_untrained_tokens = do_nothing

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
import os
os.environ["KAGGLEHUB_CACHE"] = "./downloaded_kaggle"
os.environ["KAGGLE_USERNAME"] = "atharvaingle"
os.environ["KAGGLE_KEY"] = "461b0631fca2dc582a2019918b23f72d"
import pandas as pd
import kagglehub
kagglehub.competition_download("wsdm-cup-multilingual-chatbot-arena")
import time
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
import torch.nn as nn
import torch.nn.functional as F
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

In [3]:
df = pd.read_parquet("downloaded_kaggle/competitions/wsdm-cup-multilingual-chatbot-arena/train.parquet")
df["labels"] = df["winner"].map({"model_a": 0, "model_b": 1})
df.head(2)

,id,prompt,response_a,response_b,winner,model_a,model_b,language,labels
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak,0
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian,0


In [4]:
# ckpt = "unsloth/Qwen2.5-7B-bnb-4bit"
ckpt = "unsloth/Qwen2.5-3B"
tokenizer = AutoTokenizer.from_pretrained(
        ckpt,
        use_fast=True,
        trust_remote_code=True,
        from_slow=True,
        add_prefix_space=False,
        padding_side="left",
        truncation_side="left",
    )
tokenizer.add_eos_token = True

In [5]:
def prepare_text(row):
    prompt, res_a, res_b = row["prompt"], row["response_a"], row["response_b"]
    text = "## Prompt\n" + prompt + "\n\n### Response A\n" + res_a + "\n\n### Response B\n" + res_b + "\n\n" + "## Which response is better?"
    return text

df["text"] = df.apply(prepare_text, axis=1)
df.head(1)

,id,prompt,response_a,response_b,winner,model_a,model_b,language,labels,text
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak,0,## Prompt\nvieš po Slovensky?\n\n### Response ...


In [6]:
def tokenize_func(example):
    return tokenizer(example["text"], truncation=True, max_length=1024)

In [7]:
ds = Dataset.from_pandas(df.head(5000))
ds = ds.train_test_split(test_size=0.2, seed=42, shuffle=True)
tok_ds = ds.map(tokenize_func, num_proc=2)
tok_ds

Map (num_proc=2):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'response_a', 'response_b', 'winner', 'model_a', 'model_b', 'language', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'prompt', 'response_a', 'response_b', 'winner', 'model_a', 'model_b', 'language', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [8]:
from unsloth import FastLanguageModel
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+


# model_name = "unsloth/Qwen2-7B-bnb-4bit";load_in_4bit = True
model_name = ckpt; load_in_4bit = True,

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length,
    dtype = dtype,
)

==((====))==  Unsloth 2024.12.9: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.679 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
model.lm_head = nn.Linear(in_features=model.config.hidden_size, out_features=2, bias=False)

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        # can easily be trained because it has only 2 tokens
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    modules_to_save=["lm_head"]
    # init_lora_weights = 'loftq',
    # loftq_config = LoftQConfig(loftq_bits = 4, loftq_iter = 1), # And LoftQ
)
print("trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Unsloth: Offloading output_embeddings to disk to save VRAM


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/unsloth/models/_utils.py:747: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filename, map_

Unsloth: Training lm_head in mixed precision to save VRAM
trainable parameters: 29937664


In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=16)

In [12]:
def compute_metrics(eval_preds):
    logits = eval_preds.predictions
    labels = eval_preds.label_ids
    logits = logits[:, -1]
    probs = torch.from_numpy(logits).float().softmax(-1).numpy()
    preds = probs.argmax(-1)
    acc = accuracy_score(y_true=labels, y_pred=preds)
    return {"accuracy": acc}

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs = False, num_items_in_batch = None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # print(f"logits shape before: {logits.shape}")
        logits = logits[:, -1]
        # print(f"logits shape after last token: {logits.shape}")
        # print(f"labels shape: {labels.shape}")
        # st_time = time.perf_counter()
        loss = F.cross_entropy(logits, labels)
        # print(f"time for cross_entropy calc: {time.perf_counter() - st_time}")
        return (loss, outputs) if return_outputs else loss

In [13]:
training_args = TrainingArguments(
    output_dir="outputs",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    num_train_epochs=1,
    lr_scheduler_type="cosine",
    warmup_steps=10,
    bf16=True,
    tf32=True,
    bf16_full_eval=True,
    optim="paged_adamw_8bit",
    group_by_length=True,
    gradient_checkpointing=False,
    torch_compile=False,
    use_liger_kernel=False,
)
trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tok_ds["train"],
    eval_dataset=tok_ds["test"],
    compute_metrics=compute_metrics,
)

In [14]:
# keep_cols = ["input_ids", "attention_mask"]
# sample_ds = tok_ds["train"].remove_columns(column_names=[c for c in tok_ds["train"].column_names if c not in keep_cols])
# dl = torch.utils.data.DataLoader(tok_ds["train"], batch_size=32, collate_fn=data_collator)
# batch = next(iter(dl))
# batch = {k: v.to(model.device) for k, v in batch.items()}
# with torch.no_grad():
#     out = model(**batch)

In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 125
 "-____-"     Number of trainable parameters = 29,937,664


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.917915,0.536000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [16]:
model.save_pretrained("lora_trimmed")
tokenizer.save_pretrained("lora_trimmed")

('lora_trimmed/tokenizer_config.json',
 'lora_trimmed/special_tokens_map.json',
 'lora_trimmed/vocab.json',
 'lora_trimmed/merges.txt',
 'lora_trimmed/added_tokens.json',
 'lora_trimmed/tokenizer.json')

In [17]:
trainer_stats.metrics

{'train_runtime': 1070.7782,
 'train_samples_per_second': 3.736,
 'train_steps_per_second': 0.117,
 'total_flos': 5.568576431849472e+16,
 'train_loss': 0.8533303833007813,
 'epoch': 1.0}